**<font color=black size=5>K近邻</font>**

**1** 介绍K近邻算法的基本原理

**2** 介绍K近邻算法的KDTree实现方法

**3** 通过简单案例和鸢尾花分类案例，对比分析本文算法和sklearn的结果。

In [1]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import heapq

**<font color=black size=4>1 K近邻算法</font>** 

K近邻算法多用于分类任务，本文仅就其分类问题进行讨论。**基本思想**：给定一个已知标签的数据集，对一个输入的未知标签的样本，计算其与数据集中各样本之间的距离，找出与之距离最小的$k$个样本；统计这$k$个样本的类别，以多数服从少数的原则，最终确定未知样本的分类标签。算法的三个基本要素如下：

+ **$k$值：** $k$值越大，模型越简单；反之，越复杂。试想：若$k$取数据集的样本总数$n$，那么无论输入的未知样本是什么，最终得到的其分类标签都是数据集中占比最大的那一类的标签，这样的模型是非常简单的。

+ **距离度量方法**：不同的距离度量确定的最邻近点通常是不同的。一般选择$Minkowski$距离，其定义为：$$ L_p(x_i, x_j)=(\sum_{l=1}^{m}|x_i^{(l)}- x_j^{(l)}|^p)^{\frac{1}{p}} $$
其中：$p>=1$；$m$为特征的维数；$l$为第几维。<br>
当$p=2$时，为欧氏距离：$ L_2(x_i, x_j)=(\sum_{l=1}^{m}|x_i^{(l)}- x_j^{(l)}|^2)^{\frac{1}{2}} $<br>
当$p=1$时，为曼哈顿距离：$ L_1(x_i, x_j)=\sum_{l=1}^{m}|x_i^{(l)}- x_j^{(l)}| $<br>
当$p=\infty$时，为各个维度下距离的最大值：$ L_\infty(x_i, x_j)=max|x_i^{(l)}- x_j^{(l)}| $

+ **分类决策规则**：一般是采用多数表决的规则，即由输入的未知样本的k个邻近样本中的多数类决定未知样本的类。

**<font color=black size=4>2 KDTree</font>** 

K近邻算法的思想很简单，但存在的问题是：当样本数量较大时，为预测一个未知样本的类别标签，需要遍历计算其与所有

样本之间的距离（线性搜索），这无疑会造成较大的时间开销。通过构造KDTree，减少计算距离的次数，提高算法效率。


**<font color=black size=3.5>2.1 KDTree构造</font>** 

+ **两个基本操作：**<br>
1) 计算划分维度： $ l=j \  \% \ m $。$\%$为取模运算；$j$的起始值为0，每完成一次划分$j=j+1$；$m$为特征的维数<br>
2) 计算划分点： 按$l$维的值对$X$从小到大排序，取中间点作为划分点。排序后中间点索引的计算方法为：$len(X)\ // \ 2$<br>
+ **示例：** 给定数据集$X = [[2, 3], [5, 4], [9, 6], [4, 7], [8, 1], [7, 2]]$，构造一个kd树。<br>
1) 第1次划分： <br>
$ l_1=0 \  \% \ 2 = 0$，也就是划分维度为第0维；<br>
按第0维元素值对$X$排序的结果为$X = [[2, 3], [4, 7], [5, 4], [7, 2], [8, 1], [9, 6]]$；<br>
划分点的索引为$len(X)\ // \ 2 = 6 \ // \ 2 = 3$，对应$[7, 2]$，存储到根节点；
将$[7, 2]$左侧的点存储到左子树，右侧的点存储到右子树。<br>
<img src="../src/kdtree1.png" width=280, heigth=200>
2) 第2次划分： <br>
$j = j + 1$，即$ l_2=1 \  \% \ 2 = 1$；<br>
对第1次划分得到的左右子树，分别进行同样的划分操作，得到新的左右子树。<br>
<img src="../src/kdtree2.png" width=280, heigth=200>
3) 第3次划分： <br>
$j = j + 1$，即$ l_3=2 \  \% \ 2 = 0$；<br>
划分后发现所有叶节点中只有一个点，因而无法继续划分，划分停止，得到最终的KDTree。
<img src="../src/kdtree3.png" width=280, heigth=200><br>
<font color=red>注：可以发现上述过程，划分维度是交替进行的，因此可以称为交替维度划分法。除此之外，还可以
采用最大方差法，即：每一次划分计算各个维度上的方差，取方差最大的维度作为该次的划分维度。</font>

**<font color=black size=3.5>2.2 KDTree搜索</font>** <br>
+ 上述的二维数据集构造的KDTree可以对应用如下二维平面的划分来表达，每一次划分即对应在该次的划分维度上根据划分点将平面切割。如下图所示，第$1$次划分在$l=0$维度的$x^{(1)}=7$处将平面切割为左右两个区域。
<img src="../src/kdtree4.png" width=280, heigth=200><br>
+ **搜索过程**<br>
以寻找点(2.5, 4.5)的最邻近点为例：<br>
1) 按上述创建的KDTree从根节点开始判断，直至达到叶节点，可以得到(2.5, 4.5)的第一个候选最邻近点为(4, 7)，对应的欧式距离$d_{min}$为2.915；<br>
2) 回溯。(4, 7)的根节点为(5, 4)，计算(2.5, 4.5)与(5, 4)的距离为2.550<2.915，因此(2.5, 4.5)的最邻近点更新为(5, 4)，$d_{min}$更新为2.550；由于不清楚(5, 4)的左子树中有没有更近的点，因此需要做一步判断：计算(5, 4)的划分维度$l$下，(2.5, 4.5)与(5, 4)在$l$上的距离$d_l$，若$d_l<d_{min}$，则需要进入(5, 4)的右子树重复1)、2)的搜索过程，反之则不需要；在本例中，(5, 4)的划分维度$l=1$，则$d_l=|4.5-4|=0.5<2.550$，因此进入左子树进行搜索，计算(2.5, 4.5)与(2, 3)的距离为1.581，因此更新最邻近点为(2, 3)，$d_{min}=1.581$。<br>
<img src="../src/kdtree5.png" width=280, heigth=200><br>
3)继续向上回溯。(5, 4)的根节点为(7, 2)，计算(2.5, 4.5)与(7, 2)的距离为5.148>1.581，因此不更新最邻近点与$d_{min}$。同样地，判断$d_l$和$d_{min}$的大小关系，发现$d_l= 4.5 > d_{min}=1.581$，因此不需要再进入
(7, 2)的右子树进行搜索。本例中(7, 2)为整体树的根节点，因此到此搜索也就停止，最终得到的最邻近点为(2, 3)，最邻近距离为1.581。<br>

In [2]:
class Node(object):
    """kd树中的一个节点"""

    def __init__(self, seg_axis, seg_point, left=None, right=None):
        self.seg_axis = seg_axis  # int 划分维度
        self.seg_point = seg_point  # list 划分点
        self.left = left  # class 左子树
        self.right = right  # class 右子树

In [3]:
class KDTree(object):
    """kd树类"""

    def __init__(self, X):
        self.X = X

        def create(X, depth=0):
            """递归创建kd树
            Parameters
            ----------
            X: list 样本
            depth: int 树的深度 默认为0
            """
            if not X:
                return

            # 交替轴法确定划分维度
            k = len(X[0])
            ax = depth % k
            sort_X = sorted(X, key=lambda x: x[ax])
            mid = len(X) // 2

            return Node(seg_axis=ax,
                        seg_point=sort_X[mid],
                        left=create(sort_X[:mid], depth + 1),
                        right=create(sort_X[mid + 1:], depth + 1))

        self.root = create(X, 0)

    def query(self, x, k=1, p=2):
        """kd树查询目标点的k个最邻近点
        Parameters
        ----------
        x: list 目标点
        k: int 最邻近点个数 默认为1
        p: int 距离度量方法 默认为2 代表欧式距离

        Return
        ------
        dist: list k个最邻近点距离（由大到小排列）
        ind: list 对应dist的k个最邻近点在X中的索引
        """
        self.knn = [(-np.inf, None)] * k

        def search_knn(node):
            """query方法的辅助方法。用于访问kd树的结点，并计算结点与目标的距离，
            确定k个最邻近的点。
            Parameters
            ----------
            node: class kd树的结点
            """
            if node is not None:
                ax = node.seg_axis
                ax_diff = abs(x[ax]-node.seg_point[ax])  # 目标点和结点在ax维度下的距离

                if x[ax] < node.seg_point[ax]:
                    search_knn(node.left)
                else:
                    search_knn(node.right)

                # 计算目标点与结点的距离
                d = np.linalg.norm(np.array(x)-np.array(node.seg_point), p)

                # 利用堆结构存储距离和划分点信息
                heapq.heappushpop(self.knn, (-d, node.seg_point))

                # 回溯过程中，是否需要进入另一子区域的判断
                if ax_diff < -self.knn[0][0]:
                    if x[ax] < node.seg_point[ax]:
                        search_knn(node.right)
                    else:
                        search_knn(node.left)

        search_knn(self.root)
        self.knn.sort(reverse=True)  # 对k个邻近点进行排序（也可不加这行代码）
        
        dist = []
        ind = []
        for item in self.knn:
            dist.append(abs(item[0]))
            ind.append(self.X.index(item[1]))

        return dist, ind

In [4]:
class MyKNeighborsClassifier(KDTree):
    """K近邻分类器"""

    def __init__(self, X, y, k=1, p=2):
        super(MyKNeighborsClassifier, self).__init__(X)  # 继承父类的构造方法
        self.y = y
        self.k = k
        self.p = p

    def kneighbors(self, x, k=1):
        """获取目标点的k个最邻近点"""
        neigh_dist, neigh_ind = self.query(x, k)

        return neigh_dist, neigh_ind

    def predict(self, x):
        """预测目标点的分类标签"""
        _, neigh_ind = self.query(x, self.k)

        # 多数服从少数的投票方法
        vote = {}
        for i in neigh_ind:
            vote[self.y[i]] = vote.get(self.y[i], 0) + 1
        out = sorted(vote.items(), key=lambda s: s[1], reverse=True)

        return out[0][0]

**<font color=black size=4>3 案例</font>** 

**<font color=black size=3.5>3.1 简单案例</font>** 

In [5]:
# 数据准备
X = [[2, 3], [5, 4], [9, 6], [4, 7], [8, 1], [7, 2]]
y = [0, 0, 1, 1, 2, 2]

In [6]:
# 本文算法
clf = MyKNeighborsClassifier(X, y, k=3)
neigh_dist, neigh_ind = clf.kneighbors([2.5, 4.5], k=3)
print('本文算法')
print('-------')
print('最邻近点的距离：', neigh_dist)
print('最邻近点的索引：', neigh_ind)
print('预测类别：', clf.predict([2.5, 4.5]))

本文算法
-------
最邻近点的距离： [1.5811388300841898, 2.5495097567963922, 2.9154759474226504]
最邻近点的索引： [0, 1, 3]
预测类别： 0


In [7]:
# sklearn
neigh = KNeighborsClassifier(n_neighbors=3, algorithm='kd_tree')
neigh.fit(X, y)
dist, ind = neigh.kneighbors(np.array([[2.5, 4.5]]), n_neighbors=3)
print('sklearn')
print('-------')
print('最邻近点的距离：', dist)
print('最邻近点的索引：', ind)
print('预测类别：', neigh.predict([[2.5, 4.5]]))

sklearn
-------
最邻近点的距离： [[1.58113883 2.54950976 2.91547595]]
最邻近点的索引： [[0 1 3]]
预测类别： [0]


**<font color=black size=3.5>3.2 鸢尾花分类</font>** <br>

In [8]:
# 数据准备
iris = load_iris()
X2 = iris.data
y2 = iris.target

In [9]:
# 本文算法
X2_copy = X2.copy().tolist()  # 转换成本文算法要求的数据类型
y2_copy = y2.copy().tolist()

clf2 = MyKNeighborsClassifier(X2_copy, y2_copy, k=3)
neigh_dist2, neigh_ind2 = clf2.kneighbors([2, 3, 2, 3], 3)
print('本文算法')
print('-------')
print('最邻近点的距离：', neigh_dist2)
print('最邻近点的索引：', neigh_ind2)
print('预测类别：', clf2.predict([2, 3, 2, 3]))

本文算法
-------
最邻近点的距离： [3.7376463182061515, 3.753664875824692, 3.7589892258425004]
最邻近点的索引： [8, 38, 42]
预测类别： 0


In [10]:
# sklearn
neigh2 = KNeighborsClassifier(n_neighbors=3, algorithm='kd_tree')
neigh2.fit(X2, y2)
dist2, ind2 = neigh2.kneighbors(np.array([[2, 3, 2, 3]]), n_neighbors=3)
print('sklearn')
print('-------')
print('最邻近点的距离：', dist2)
print('最邻近点的索引：', ind2)
print('预测类别：', neigh2.predict([[2, 3, 2, 3]]))

sklearn
-------
最邻近点的距离： [[3.73764632 3.75366488 3.75898923]]
最邻近点的索引： [[ 8 38 42]]
预测类别： [0]


可以看出：本文算法与sklearn算法得到的结果一致。

<font color=red>*注：本文重在通过代码理解算法的工作原理，仅供学习使用。更多关于算法相关的资料可以在参考里找到。</font>

**<font color=black size=4>参考</font>**

1 李航. (2012) 统计学习方法. 清华大学出版社, 北京.

2 [sklearn文档](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)